# Aim of this notebook

Proof of concept for a collaborative filtering based recommendation system using Numpy. I plan to implement the model on my own here, and use gradient descent for learning the objective function.

## Non-Weighted Matrix Factorization

For a first try, I will try to implement a non-weighted matrix factorization <br>

3 users, 3 items, 2 features <br>
<br>
Define the user item matrix as A with values in {-1, 0, 1} such that <br>
 1 -> user likes the item <br>
 0 -> user hasn't reviewed the item <br>
-1 -> user doesn't like the item <br>

In [3]:
import numpy as np

#taking the example from https://towardsdatascience.com/recommender-systems-in-python-from-scratch-643c8fc4f704
#
# |U1F1 U1F2 |    |F1I1 F1I2 F1I3 |     |U1I1 U1I2 U1I3|
# |U2F1 U2F2 | *  |F2I1 F2I2 F2I3 |  =  |U2I1 U2I2 U2I3|
# |U3F1 U3F3 |                          |U3I1 U3I2 U3I3|
#

A = np.array([(1,0,0),(-1,0,1), (-1,1,1)])